## To Do

* Posterior predictive sampling
    * Univariate families **DONE**
    * Multivariate families **IN PROGRESS**
* Model string representation **NOT STARTED**
* Model aliases **IN PROGRESS**
    * Aliases work for predictors of both non-distributional and distributional models **DONE**
    * Make sure alias of response name is used in predictions **NOT STARTED**

## Examples working

* `alternative_links_binary.ipynb`
* `beta_regression.ipynb`
    * TypeError: Encountered error in ELPD computation of compare.
* `circular_regression.ipynb`
* `ESCS_multiple_regression.ipynb`
    * Needs updates because of old API
* `getting_started.ipynb`
    * Needs to update the Family part
* `hierarchical_binomial_bambi.ipynb`
    * Works but needs to update a dim name
* `logistic_regression.ipynb`
* `model_comparison.ipynb`
* `multi-level_regression.ipynb`
* `negative_binomial.ipynb`
* `radon_example.ipynb`
    * xarray issues, not bambi related
* `shooter_crossed_random_ANOVA.ipynb`
* `sleepstudy.ipynb`
    * There's a xarray problem in a viz, not related to Bambi
* `splines_cherry_blossoms.ipynb`
    * Works, but needs updates because some things use old API
* `Strack_RRR_re_analysis.ipynb`
* `t_regression.ipynb`
* `t_test.ipynb`
* `wald_gamma_glm.ipynb`

## Examples not working

* `categorical_regression.ipynb`
    * Fixed posterior predictive sampling, but the order of categories is broken

In [ ]:
import bambi as bmb
import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
size = 100
true_intercept = 1
true_slope = 2

x = np.linspace(0, 1, size)
# y = a + b*x
true_regression_line = true_intercept + true_slope * x
# add noise
y = true_regression_line + np.random.normal(scale=0.5, size=size)

# Add outliers
x_out = np.append(x, [0.1, 0.15, 0.2])
y_out = np.append(y, [8, 6, 9])

data = pd.DataFrame({
    "x": x_out, 
    "y": y_out
})

In [ ]:
t_model = bmb.Model("y ~ x", data, family="t", priors={"nu": 2})
t_model.build()
t_fitted = t_model.fit(draws=500)
#t_model.predict(t_fitted, kind="pps")

In [ ]:
t_model.predict(t_fitted, kind="pps")

---

# Documentation notes

## Names

The logic is `{parent}_{child}`. The exception is the linear predictor of the main parameter, which is only `{child}`.

```python
bmb.Formula(
    "y ~ 1 + x + z",
    "sigma ~ 1 + x"
)
```

* `Intercept`
* `x`
* `z`
* `y_sigma`, the sigma parameter in the normal likelihood
* `sigma_Intercept`
* `sigma_x`

### Potential improvements

What about using just `sigma` instead of `{response_name}_sigma`. The gain: cleaner names. The downside: This pattern could be a conflict in a small number of cases where the name of predictors match the name of parameters in the likelihood. It could be "fixed" easily, by double checking the names of the predictors before creating the model.

If we want to create markdown to later be used as source of quarto documents...

* https://stackoverflow.com/questions/36237477/python-docstrings-to-github-readme-md
* https://pypi.org/project/pydoc-markdown/

* BRING CONSTANT COMPONENT NAMES BACK
    * Yes or No?...

In [4]:
from bambi import Model, load_data
import numpy as np

Migration guide... 

* What used to be `model.terms["name"]` is now `model.response_component.terms["name"]`.
* Correlated priors for group-specific terms are not supported anymore. They will be brought back in the future.

To Do... 

* A None link when it's not specified. This will allow us to have

```python
likelihood = Likelihood("Normal", params=["mu", "sigma"], parent="mu")
Family("MyFamily", likelihood=likelihood, link="log")
```

which means the non-parent parameter is non-distributional and thus it does not make sense to have a link funciton for it.

```python
test_posterior_predictive_categorical
test_predict_multinomial
test_posterior_predictive_multinomial
```